In [23]:
import pandas as pd
from flask import Flask, request, jsonify
import joblib
import numpy as np

In [24]:
# Initialize Flask app
app = Flask(__name__)

# Load the trained model
model_path = 'model.pkl'
classifier = joblib.load(model_path)

In [25]:
@app.route('/predict', methods=['POST'])
def predict():
    """
    Endpoint to receive JSON payload and return prediction.
    """
    try:
        # Get the JSON payload from the request
        json_payload = request.json
        
        # Validate the structure of the JSON payload
        if isinstance(json_payload, dict) and all(isinstance(value, list) for value in json_payload.values()):
            # Convert the JSON payload into a DataFrame-friendly format
            # Assuming each key in the JSON represents a column and its value is a list of values for that column
            df = pd.DataFrame.from_dict(json_payload)
            
            # Perform one-hot encoding if necessary
            # Note: This step depends on whether your model expects raw features or one-hot encoded ones
            # For example, if you have categorical features like 'Sex' and 'Embarked', you might need to encode them
            # df = pd.get_dummies(df)
            
            # Convert the DataFrame to a NumPy array and make predictions
            # Assuming the model expects a 2D array; reshape if necessary
            prediction = classifier.predict(np.array(df).reshape(1, -1))  # Reshape to match the model's input shape
            
            # Return the prediction as a JSON response
            return jsonify({'prediction': int(prediction[0])}), 200
        else:
            return jsonify({'error': 'Invalid JSON payload structure'}), 400
    except Exception as e:
        return jsonify({'error': str(e)}), 500

In [27]:
if __name__ == '__main__':
    print("Starting Flask application...")
    app.run(port=5000)


Starting Flask application...
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
127.0.0.1 - - [16/May/2024 23:12:15] "POST /predict HTTP/1.1" 200 -
